In [1]:
import warnings
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

In [2]:
warnings.filterwarnings('ignore')
House_Sub=pd.read_csv('Ames Real Estate Data.csv')
House_Main=pd.read_csv('Ames_HousePrice.csv',index_col=0)

In [3]:
House_Main['PoolQC'].isnull().sum()/len(House_Main['PoolQC'])

0.9965116279069768

In [4]:
House_Main.describe(include='all')

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
count,2.580000e+03,2580.000000,2580.000000,2580.000000,2580,2118.000000,2580.000000,2580,168,2580,...,2580.000000,2580.000000,9,525,97,2580.000000,2580.000000,2580.000000,2580,2580
unique,NaN,NaN,NaN,NaN,7,NaN,NaN,2,2,4,...,NaN,NaN,4,4,4,NaN,NaN,NaN,10,6
top,NaN,NaN,NaN,NaN,RL,NaN,NaN,Pave,Grvl,Reg,...,NaN,NaN,TA,MnPrv,Shed,NaN,NaN,NaN,WD,Normal
freq,NaN,NaN,NaN,NaN,2008,NaN,NaN,2570,105,1641,...,NaN,NaN,3,301,88,NaN,NaN,NaN,2392,2414
mean,7.148300e+08,1486.039922,178059.623256,57.691860,NaN,68.516053,10120.153488,NaN,NaN,NaN,...,16.200388,1.662016,NaN,NaN,NaN,48.731395,6.150775,2007.838372,NaN,NaN
std,1.886626e+08,488.650181,75031.089374,42.802105,NaN,22.835831,8126.937892,NaN,NaN,NaN,...,56.824783,30.339396,NaN,NaN,NaN,498.725058,2.670647,1.312333,NaN,NaN
min,5.263011e+08,334.000000,12789.000000,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,...,0.000000,0.000000,NaN,NaN,NaN,0.000000,1.000000,2006.000000,NaN,NaN
25%,5.313630e+08,1112.000000,129975.000000,20.000000,NaN,57.000000,7406.750000,NaN,NaN,NaN,...,0.000000,0.000000,NaN,NaN,NaN,0.000000,4.000000,2007.000000,NaN,NaN
50%,5.354546e+08,1436.000000,159900.000000,50.000000,NaN,68.000000,9391.000000,NaN,NaN,NaN,...,0.000000,0.000000,NaN,NaN,NaN,0.000000,6.000000,2008.000000,NaN,NaN
75%,9.071811e+08,1733.000000,209625.000000,70.000000,NaN,80.000000,11494.000000,NaN,NaN,NaN,...,0.000000,0.000000,NaN,NaN,NaN,0.000000,8.000000,2009.000000,NaN,NaN


# Find Columns whose dtypes are numeric and initial cor analysis of those columns to target: SalePrice

In [5]:
#The target dependent varibale for this case is SalePrice column in House_Main table. In order to do some initial analysis, 
#we find columns that are of numeric type first.
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
House_Main_Num=House_Main.select_dtypes(include=numerics)

In [6]:
#PID is not useful as it's working as an ID index, we drop that column
House_Main_Num=House_Main_Num.drop(['PID'],axis=1)

In [7]:
#Do the correlation analysis of the numeric variables

#set the display of output table/dataframe to max columns
pd.set_option('display.max_columns',None)

#Find the correlation of all numeric variables
cormax=round(House_Main_Num.corr(),2)

In [8]:
cormax.head()

,GrLivArea,SalePrice,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
GrLivArea,1.00,0.72,0.07,0.35,0.26,0.56,-0.11,0.23,0.31,0.39,0.16,-0.02,0.25,0.41,0.54,0.67,0.09,0.04,-0.05,0.64,0.45,0.54,0.13,0.81,0.46,0.27,0.50,0.48,0.25,0.32,0.00,0.01,0.10,0.06,-0.00,0.04,-0.01
SalePrice,0.72,1.00,-0.08,0.36,0.27,0.79,-0.10,0.54,0.51,0.50,0.46,0.02,0.17,0.65,0.64,0.26,-0.03,0.29,-0.03,0.53,0.29,0.15,-0.11,0.49,0.49,0.52,0.64,0.63,0.33,0.31,-0.12,0.03,0.12,0.03,-0.02,0.01,-0.00
MSSubClass,0.07,-0.08,1.00,-0.44,-0.20,0.06,-0.08,0.05,0.06,0.02,-0.07,-0.07,-0.11,-0.22,-0.26,0.31,0.01,-0.00,-0.01,0.14,0.18,-0.02,0.25,0.03,-0.05,0.11,-0.03,-0.09,-0.02,-0.01,-0.03,-0.04,-0.06,-0.01,-0.03,0.01,-0.03
LotFrontage,0.35,0.36,-0.44,1.00,0.46,0.18,-0.05,0.10,0.06,0.20,0.18,0.06,0.10,0.31,0.44,0.01,-0.01,0.11,-0.03,0.17,0.03,0.25,0.02,0.34,0.25,0.05,0.30,0.34,0.12,0.14,0.00,0.03,0.08,0.09,0.02,0.00,-0.00
LotArea,0.26,0.27,-0.20,0.46,1.00,0.08,-0.03,0.01,0.01,0.10,0.16,0.08,0.02,0.22,0.31,0.02,-0.00,0.12,0.03,0.12,0.03,0.14,-0.01,0.20,0.25,-0.02,0.18,0.20,0.16,0.08,0.02,0.02,0.05,0.05,0.04,0.01,-0.02


In [9]:
#From the above table we find the correlation of SalePrice and other independent numeric varibales
cormax=cormax['SalePrice']
cormax

GrLivArea        0.72
SalePrice        1.00
MSSubClass      -0.08
LotFrontage      0.36
LotArea          0.27
OverallQual      0.79
OverallCond     -0.10
YearBuilt        0.54
YearRemodAdd     0.51
MasVnrArea       0.50
BsmtFinSF1       0.46
BsmtFinSF2       0.02
BsmtUnfSF        0.17
TotalBsmtSF      0.65
1stFlrSF         0.64
2ndFlrSF         0.26
LowQualFinSF    -0.03
BsmtFullBath     0.29
BsmtHalfBath    -0.03
FullBath         0.53
HalfBath         0.29
BedroomAbvGr     0.15
KitchenAbvGr    -0.11
TotRmsAbvGrd     0.49
Fireplaces       0.49
GarageYrBlt      0.52
GarageCars       0.64
GarageArea       0.63
WoodDeckSF       0.33
OpenPorchSF      0.31
EnclosedPorch   -0.12
3SsnPorch        0.03
ScreenPorch      0.12
PoolArea         0.03
MiscVal         -0.02
MoSold           0.01
YrSold          -0.00
Name: SalePrice, dtype: float64

In [10]:
#We sort the correlation from highest to lowest
np.abs(cormax).sort_values(ascending=False)

SalePrice        1.00
OverallQual      0.79
GrLivArea        0.72
TotalBsmtSF      0.65
1stFlrSF         0.64
GarageCars       0.64
GarageArea       0.63
YearBuilt        0.54
FullBath         0.53
GarageYrBlt      0.52
YearRemodAdd     0.51
MasVnrArea       0.50
Fireplaces       0.49
TotRmsAbvGrd     0.49
BsmtFinSF1       0.46
LotFrontage      0.36
WoodDeckSF       0.33
OpenPorchSF      0.31
BsmtFullBath     0.29
HalfBath         0.29
LotArea          0.27
2ndFlrSF         0.26
BsmtUnfSF        0.17
BedroomAbvGr     0.15
ScreenPorch      0.12
EnclosedPorch    0.12
KitchenAbvGr     0.11
OverallCond      0.10
MSSubClass       0.08
3SsnPorch        0.03
PoolArea         0.03
BsmtHalfBath     0.03
LowQualFinSF     0.03
BsmtFinSF2       0.02
MiscVal          0.02
MoSold           0.01
YrSold           0.00
Name: SalePrice, dtype: float64

# Find columns whose dtypes are non-numeric and label-encode or dummify variables in those columns

In [11]:
#Get all the columns whose dtypes are not numeric, i.e. object
House_Main_Nonnum=House_Main.select_dtypes(exclude=numerics)
House_Main_Nonnum.dtypes

MSZoning         object
Street           object
Alley            object
LotShape         object
LandContour      object
Utilities        object
LotConfig        object
LandSlope        object
Neighborhood     object
Condition1       object
Condition2       object
BldgType         object
HouseStyle       object
RoofStyle        object
RoofMatl         object
Exterior1st      object
Exterior2nd      object
MasVnrType       object
ExterQual        object
ExterCond        object
Foundation       object
BsmtQual         object
BsmtCond         object
BsmtExposure     object
BsmtFinType1     object
BsmtFinType2     object
Heating          object
HeatingQC        object
CentralAir       object
Electrical       object
KitchenQual      object
Functional       object
FireplaceQu      object
GarageType       object
GarageFinish     object
GarageQual       object
GarageCond       object
PavedDrive       object
PoolQC           object
Fence            object
MiscFeature      object
SaleType        

In [12]:
House_Main_Nonnum.shape[1] #We know there are 43 variables whose dtypes are 'object'

43

In [13]:
House_Main_Nonnum.isnull().sum(axis=0) #From the list below, we know some object variables have null-type values.

MSZoning            0
Street              0
Alley            2412
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType         14
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           69
BsmtCond           69
BsmtExposure       71
BsmtFinType1       69
BsmtFinType2       70
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
KitchenQual         0
Functional          0
FireplaceQu      1241
GarageType        127
GarageFinish      129
GarageQual        129
GarageCond        129
PavedDrive          0
PoolQC           2571
Fence            2055
MiscFeature      2483
SaleType            0
SaleCondition       0
dtype: int64

In [14]:
#Subset the House_Main_Nonnum columns that do not have nan values
House_Main_Nonnum_Init=House_Main_Nonnum[House_Main_Nonnum.columns[~House_Main_Nonnum.isnull().any()]]

House_Main_Nonnum_Init.isnull().sum(axis=0)

MSZoning         0
Street           0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
ExterQual        0
ExterCond        0
Foundation       0
Heating          0
HeatingQC        0
CentralAir       0
KitchenQual      0
Functional       0
PavedDrive       0
SaleType         0
SaleCondition    0
dtype: int64

In [15]:
#We do a value counts here to make sure words are not used to represent numbers (like 'One', 'Two', ...) If there are we 
#could just replace with integers. (The result shows none)
House_Main_Nonnum_Init.value_counts

<bound method DataFrame.value_counts of     MSZoning Street LotShape LandContour Utilities LotConfig LandSlope  \
1         RL   Pave      Reg         Lvl    AllPub    Corner       Gtl   
2         RL   Pave      Reg         Lvl    AllPub    Inside       Gtl   
3    C (all)   Pave      Reg         Lvl    AllPub    Inside       Gtl   
4         RL   Pave      Reg         Lvl    AllPub    Corner       Gtl   
5         RL   Pave      Reg         Lvl    AllPub    Inside       Gtl   
..       ...    ...      ...         ...       ...       ...       ...   
759       RL   Pave      Reg         Lvl    AllPub    Inside       Gtl   
760       RL   Pave      IR1         Lvl    AllPub   CulDSac       Gtl   
761       RH   Pave      Reg         HLS    AllPub    Inside       Gtl   
762       RL   Pave      Reg         Lvl    AllPub    Inside       Gtl   
763       RL   Pave      IR1         Lvl    AllPub    Inside       Gtl   

    Neighborhood Condition1 Condition2 BldgType HouseStyle RoofStyle Ro

In [16]:
#Making sure there's no empty cells in the columns of House_Main_Nonnum_Init
for col in House_Main_Nonnum_Init:
    print(col,(House_Main_Nonnum_Init[col].values=='').sum())

MSZoning 0
Street 0
LotShape 0
LandContour 0
Utilities 0
LotConfig 0
LandSlope 0
Neighborhood 0
Condition1 0
Condition2 0
BldgType 0
HouseStyle 0
RoofStyle 0
RoofMatl 0
Exterior1st 0
Exterior2nd 0
ExterQual 0
ExterCond 0
Foundation 0
Heating 0
HeatingQC 0
CentralAir 0
KitchenQual 0
Functional 0
PavedDrive 0
SaleType 0
SaleCondition 0


In [17]:
for col in House_Main_Nonnum_Init:
    print(col,House_Main_Nonnum_Init[col].unique())

MSZoning ['RL' 'C (all)' 'RM' 'FV' 'RH' 'I (all)' 'A (agr)']
Street ['Pave' 'Grvl']
LotShape ['Reg' 'IR1' 'IR3' 'IR2']
LandContour ['Lvl' 'Bnk' 'HLS' 'Low']
Utilities ['AllPub' 'NoSewr']
LotConfig ['Corner' 'Inside' 'FR2' 'CulDSac' 'FR3']
LandSlope ['Gtl' 'Mod' 'Sev']
Neighborhood ['SWISU' 'Edwards' 'IDOTRR' 'OldTown' 'NWAmes' 'Blmngtn' 'Mitchel'
 'NridgHt' 'NAmes' 'CollgCr' 'SawyerW' 'MeadowV' 'BrDale' 'Gilbert'
 'Timber' 'Somerst' 'BrkSide' 'NoRidge' 'Sawyer' 'Crawfor' 'Greens'
 'ClearCr' 'GrnHill' 'StoneBr' 'Veenker' 'NPkVill' 'Blueste' 'Landmrk']
Condition1 ['Norm' 'RRAn' 'PosN' 'Artery' 'RRAe' 'Feedr' 'PosA' 'RRNn' 'RRNe']
Condition2 ['Norm' 'PosN' 'Feedr' 'Artery' 'PosA' 'RRNn' 'RRAn' 'RRAe']
BldgType ['1Fam' 'TwnhsE' '2fmCon' 'Twnhs' 'Duplex']
HouseStyle ['1Story' '2Story' 'SFoyer' 'SLvl' '1.5Fin' '2.5Unf' '1.5Unf' '2.5Fin']
RoofStyle ['Gable' 'Hip' 'Mansard' 'Gambrel' 'Flat' 'Shed']
RoofMatl ['CompShg' 'Tar&Grv' 'WdShake' 'WdShngl' 'Metal' 'Roll' 'Membran']
Exterior1st ['Wd Sdn

In [18]:
for col in House_Main_Nonnum_Init:
    print(House_Main_Nonnum_Init[col].value_counts())

RL         2008
RM          418
FV          113
RH           21
C (all)      17
I (all)       2
A (agr)       1
Name: MSZoning, dtype: int64
Pave    2570
Grvl      10
Name: Street, dtype: int64
Reg    1641
IR1     855
IR2      71
IR3      13
Name: LotShape, dtype: int64
Lvl    2337
HLS      96
Bnk      95
Low      52
Name: LandContour, dtype: int64
AllPub    2578
NoSewr       2
Name: Utilities, dtype: int64
Inside     1886
Corner      439
CulDSac     164
FR2          78
FR3          13
Name: LotConfig, dtype: int64
Gtl    2458
Mod     109
Sev      13
Name: LandSlope, dtype: int64
NAmes      410
CollgCr    236
OldTown    213
Edwards    165
Somerst    143
Gilbert    143
Sawyer     139
NWAmes     123
NridgHt    121
SawyerW    113
Mitchel    104
BrkSide    103
Crawfor     93
IDOTRR      76
NoRidge     67
Timber      54
StoneBr     43
SWISU       42
ClearCr     40
MeadowV     34
BrDale      29
Veenker     23
Blmngtn     23
NPkVill     22
Blueste     10
Greens       8
GrnHill      2
Landmrk 

In [19]:
#From the above tables we could handle different columns as follows:
# MSZoning: Group C, I, A into 'non-residential'
# HeatingQC, KitchenQual: Group Fa, Po into 'Ba' as 'Below Average'
Drop=['Street', 'Utilities','Condition2', 'RoofMatl', 'SaleType', 'SaleCondition']
# Keep: LandContour, BldgType, ExterQual, ExterCond, Foundation, PavedDrive
# LotConfig: Group FR2 and FR3 into 'Frontage'
# Convert to binary: LotShape, LandSlope, Condition1, Heating, CentralAir, Functional
# Use .75 Threshold: Neiborhood, Exterior1st, Exterior2nd
# Use .90 Threshold: HouseStyle, RoofStyle

In [20]:
House_Main_Nonnum_Init.shape

(2580, 27)

In [21]:
#Dropping columns
House_Main_Nonnum_Init=House_Main_Nonnum_Init.drop(Drop,axis=1)
House_Main_Nonnum_Init.shape

(2580, 21)

In [22]:
#MSZoning: Group C, I, A into 'non-residential'
#House_Main_Nonnum_Init['MSZoning']=
House_Main_Nonnum_Init['MSZoning']=House_Main_Nonnum_Init['MSZoning'].apply(lambda x: 
                                         'Non-residential' if x=='C (all)'
                                         else 'Non-residential' if x=='I (all)' 
                                         else 'Non-residential' if x=='A (agr)' 
                                         else 'RL' if x=='RL' 
                                         else 'RM' if x=='RM'
                                         else 'FV' if x=='FV'
                                         else 'RH')           

In [23]:
# HeatingQC, KitchenQual: Group Fa, Po into 'Ba' as 'Below Average'
House_Main_Nonnum_Init['HeatingQC']=House_Main_Nonnum_Init['HeatingQC'].str.replace('Fa','Ba').str.replace('Po','Ba')

In [24]:
# LotConfig: Group FR2 and FR3 into 'Frontage'
for i in range(len(House_Main_Nonnum_Init['LotConfig'])):
    if (House_Main_Nonnum_Init['LotConfig'].iloc[i] == 'FR2') or (House_Main_Nonnum_Init['LotConfig'].iloc[i] == 'FR3'):
        House_Main_Nonnum_Init['LotConfig'].iloc[i]='Frontage'

In [25]:
# Convert to binary: LotShape, LandSlope, Condition1, Heating, CentralAir, Functional

#LotShape:
House_Main_Nonnum_Init['LotShape']=House_Main_Nonnum_Init['LotShape'].map(lambda x: 1 if x=='Reg' else 0)
House_Main_Nonnum_Init['LotShape'].value_counts()

1    1641
0     939
Name: LotShape, dtype: int64

In [26]:
#LandSlope:
House_Main_Nonnum_Init['LandSlope']=House_Main_Nonnum_Init['LandSlope'].map(lambda x: 1 if x=='Gtl' else 0)
House_Main_Nonnum_Init['LandSlope'].value_counts()

1    2458
0     122
Name: LandSlope, dtype: int64

In [27]:
#Condition1
House_Main_Nonnum_Init['Condition1']=House_Main_Nonnum_Init['Condition1'].map(lambda x: 1 if x=='Norm' else 0)
House_Main_Nonnum_Init['Condition1'].value_counts()

1    2228
0     352
Name: Condition1, dtype: int64

In [28]:
#Heating
House_Main_Nonnum_Init['Heating']=House_Main_Nonnum_Init['Heating'].map(lambda x: 1 if ((x=='GasA') or (x=='GasW')) else 0)
House_Main_Nonnum_Init['Heating'].value_counts()

1    2565
0      15
Name: Heating, dtype: int64

In [29]:
#CentralAir
House_Main_Nonnum_Init['CentralAir']=House_Main_Nonnum_Init['CentralAir'].map(lambda x: 1 if x=='Y' else 0)
House_Main_Nonnum_Init['CentralAir'].value_counts()

1    2417
0     163
Name: CentralAir, dtype: int64

In [30]:
#Functional
House_Main_Nonnum_Init['Functional']=House_Main_Nonnum_Init['Functional'].map(lambda x: 1 if x=='Typ' else 0)
House_Main_Nonnum_Init['Functional'].value_counts()

1    2399
0     181
Name: Functional, dtype: int64

In [31]:
#Use .75 Threshold: Neiborhood, Exterior1st, Exterior2nd

def cumulatively_categorise(column,threshold=0.75):
        #Find the threshold value using the percentage and number of observations in the column
        threshold_value=int(threshold*len(column))
        #Initialise an empty list for our new minimised categories
        categories_list=[]
        #Initialise a variable to calculate the sum of frequencies
        s=0
        #Create a counter dictionary of the form unique_value: frequency
        counts=Counter(column)

        #Loop through the category name and its corresponding frequency after sorting the categories by descending order of frequency
        for i,j in counts.most_common():
        #Add the frequency to the global sum
          s+=dict(counts)[i]
        #Append the category name to the list
          categories_list.append(i)
        #Check if the global sum has reached the threshold value, if so break the loop
          if s>=threshold_value:
                break
        
        #Replace all instances not in our new categories by Other  
        column=column.apply(lambda x: x if x in categories_list else 'Other')
        
        return column

In [32]:
#Call the function with a default threshold of 75%

Treshold_75=['Neighborhood','Exterior1st', 'Exterior2nd']
for col in Treshold_75:
    House_Main_Nonnum_Init[col]=cumulatively_categorise(House_Main_Nonnum_Init[col])
print(House_Main_Nonnum_Init['Neighborhood'].value_counts())
print(House_Main_Nonnum_Init['Exterior1st'].value_counts())
print(House_Main_Nonnum_Init['Exterior2nd'].value_counts())

Other      567
NAmes      410
CollgCr    236
OldTown    213
Edwards    165
Somerst    143
Gilbert    143
Sawyer     139
NWAmes     123
NridgHt    121
SawyerW    113
Mitchel    104
BrkSide    103
Name: Neighborhood, dtype: int64
VinylSd    868
Other      504
HdBoard    416
MetalSd    407
Wd Sdng    385
Name: Exterior1st, dtype: int64
VinylSd    856
Other      571
MetalSd    404
HdBoard    380
Wd Sdng    369
Name: Exterior2nd, dtype: int64


In [33]:
#Call the function with a default threshold of 90%

Treshold_90=['HouseStyle', 'RoofStyle']
for col in Treshold_90:
    House_Main_Nonnum_Init[col]=cumulatively_categorise(House_Main_Nonnum_Init[col],threshold=0.90)
print(House_Main_Nonnum_Init['HouseStyle'].value_counts())
print(House_Main_Nonnum_Init['RoofStyle'].value_counts())

1Story    1288
2Story     773
1.5Fin     284
Other      235
Name: HouseStyle, dtype: int64
Gable    2059
Hip       473
Other      48
Name: RoofStyle, dtype: int64


In [34]:
#Dummy-Encoding Process:

Binary=['LotShape', 'LandSlope', 'Condition1', 'Heating', 'CentralAir', 'Functional']
Dummy=pd.get_dummies(House_Main_Nonnum_Init.loc[:,~House_Main_Nonnum_Init.columns.isin(Binary)],drop_first=True)
House_Main_Nonnum_Init=House_Main_Nonnum_Init[Binary]
House_Main_Nonnum_Init=pd.concat([House_Main_Nonnum_Init,Dummy],axis=1)
House_Main_Nonnum_Init.columns

Index(['LotShape', 'LandSlope', 'Condition1', 'Heating', 'CentralAir',
       'Functional', 'MSZoning_Non-residential', 'MSZoning_RH', 'MSZoning_RL',
       'MSZoning_RM', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl',
       'LotConfig_CulDSac', 'LotConfig_Frontage', 'LotConfig_Inside',
       'Neighborhood_CollgCr', 'Neighborhood_Edwards', 'Neighborhood_Gilbert',
       'Neighborhood_Mitchel', 'Neighborhood_NAmes', 'Neighborhood_NWAmes',
       'Neighborhood_NridgHt', 'Neighborhood_OldTown', 'Neighborhood_Other',
       'Neighborhood_Sawyer', 'Neighborhood_SawyerW', 'Neighborhood_Somerst',
       'BldgType_2fmCon', 'BldgType_Duplex', 'BldgType_Twnhs',
       'BldgType_TwnhsE', 'HouseStyle_1Story', 'HouseStyle_2Story',
       'HouseStyle_Other', 'RoofStyle_Hip', 'RoofStyle_Other',
       'Exterior1st_MetalSd', 'Exterior1st_Other', 'Exterior1st_VinylSd',
       'Exterior1st_Wd Sdng', 'Exterior2nd_MetalSd', 'Exterior2nd_Other',
       'Exterior2nd_VinylSd', 'Exterior2nd_Wd Sdn

In [35]:
House_Main_Nonnum.isnull().sum(axis=0).sort_values(ascending=False)

PoolQC           2571
MiscFeature      2483
Alley            2412
Fence            2055
FireplaceQu      1241
GarageCond        129
GarageQual        129
GarageFinish      129
GarageType        127
BsmtExposure       71
BsmtFinType2       70
BsmtFinType1       69
BsmtQual           69
BsmtCond           69
MasVnrType         14
Electrical          1
Functional          0
KitchenQual         0
CentralAir          0
HeatingQC           0
Heating             0
PavedDrive          0
SaleType            0
MSZoning            0
Street              0
Condition2          0
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
BldgType            0
Foundation          0
HouseStyle          0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
ExterQual           0
ExterCond           0
SaleCondition       0
dtype: int64

In [36]:
#Create another table that are categorical variables but contain NaN values

#Cat_Drop=['PoolQC','Alley','FireplaceQu','Fence','MiscFeature']

In [37]:
#Dropping some numerical/Categorical columns because they have too many NaN values and NaN doesn't mean 
#anything in those columns:
Num_Drop=['BsmtUnfSF']
House_Main_Num=House_Main_Num.drop(Num_Drop,axis=1)

In [38]:
#Merge tables to create a ML table
ML_df=pd.concat([House_Main_Nonnum_Init,House_Main_Num],axis=1)

In [39]:
ML_df_x=ML_df.loc[:,~ML_df.columns.isin(['SalePrice'])]
ML_df_y=ML_df['SalePrice']

# Time to build Pipeline for our model!

In [40]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split

In [41]:
#Now we find all NaN values before we build our model
NaN=[]
for col in ML_df_x.columns:
    if ML_df_x[col].isnull().sum(axis=0)>0:
        NaN.append(col)

In [42]:
for i in NaN:
    print(i,ML_df_x[i].values)

LotFrontage [nan 42. 60. ... 82. nan 80.]
MasVnrArea [  0. 149.   0. ...   0. 144. 125.]
BsmtFinSF1 [238. 552. 737. ... 284. 841. 380.]
BsmtFinSF2 [  0. 393.   0. ...   0.   0.   0.]
TotalBsmtSF [ 856. 1049.  837. ... 1001.  985.  777.]
BsmtFullBath [1. 1. 0. ... 0. 1. 0.]
BsmtHalfBath [0. 0. 0. ... 0. 0. 0.]
GarageYrBlt [1939. 1984. 1930. ... 1949. 2000. 1993.]
GarageCars [2. 1. 1. ... 3. 2. 2.]
GarageArea [399. 266. 216. ... 871. 486. 471.]


In [47]:
print(ML_df_x['BsmtFullBath'].unique())

[ 1.  0.  2.  3. nan]


In [54]:
from sklearn.impute import SimpleImputer #iteratative imputer
Missing=SimpleImputer(strategy='mean')
Missing=Missing.fit(ML_df_x)
ML_df_x=Missing.transform(ML_df_x.values)

In [58]:
ML_df_x

array([[1.000e+00, 1.000e+00, 1.000e+00, ..., 0.000e+00, 3.000e+00,
        2.010e+03],
       [1.000e+00, 1.000e+00, 1.000e+00, ..., 0.000e+00, 2.000e+00,
        2.009e+03],
       [1.000e+00, 1.000e+00, 1.000e+00, ..., 0.000e+00, 1.100e+01,
        2.007e+03],
       ...,
       [1.000e+00, 1.000e+00, 1.000e+00, ..., 0.000e+00, 8.000e+00,
        2.007e+03],
       [1.000e+00, 1.000e+00, 1.000e+00, ..., 0.000e+00, 7.000e+00,
        2.007e+03],
       [0.000e+00, 1.000e+00, 1.000e+00, ..., 0.000e+00, 9.000e+00,
        2.006e+03]])

In [45]:
X_train, X_test, y_train, y_test = train_test_split(ML_df_x, ML_df_y, test_size=0.3, random_state=42)
N_alpha = 100
N_rho   = 10
alphaRange = np.logspace(-10, 2, N_alpha)
rhoRange   = np.linspace(0.1,1, N_rho)
elasticnet = ElasticNet(max_iter=1000,normalize=True)
pipe = Pipeline(
    [
    ("standardize", StandardScaler()),
    ("elasticnet", elasticnet)        
    ]
)

params = {
    "elasticnet__alpha": [i for i in alphaRange],
    "elasticnet__l1_ratio": [i for i in rhoRange]
}

skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

gs = GridSearchCV(pipe, params, scoring="r2", cv=skfold)

start= time.time()
gs.fit(X_train, y_train)
print(time.time() - start)

print("The best parameters are: ", gs.best_params_)
print("The score of the final model: ", gs.score(X_test, y_test))

In [ ]:
House_Main_Nonnum_Init.shape